# 🔧 LOGISTICS CONTROL — Debug & Testing

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                 FRÉGATE 02_LOGISTICS — CONTROL PANEL                         ║
║                 Debug • Test Socketing • Inspect • Preview                   ║
╚══════════════════════════════════════════════════════════════════════════════╝
```

**Objectif**: Tester individuellement chaque composant du pipeline LOGISTICS.

---

## 📦 Cellule 1: Setup & Mount Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Configuration
DRIVE_ROOT = "/content/drive/MyDrive/DRIVE_EXODUS_V2"
UNIT_ROOT = f"{DRIVE_ROOT}/02_LOGISTICS_DEPOT"
CODEBASE = f"{UNIT_ROOT}/CODEBASE"

# Add codebase to path
import sys
sys.path.insert(0, CODEBASE)

print(f"✓ Drive monté")
print(f"✓ Unit Root: {UNIT_ROOT}")
print(f"✓ Codebase: {CODEBASE}")

In [ ]:
# Install dependencies
!pip install -q numpy

# Install Blender 4.0 if not present
import os
from pathlib import Path

BLENDER_PATH = f"{DRIVE_ROOT}/EXODUS_AI_MODELS/blender-4.0.0-linux-x64/blender"

if not Path(BLENDER_PATH).exists():
    print("⚠ Blender 4.0 non trouvé, installation en cours...")
    !wget -q https://download.blender.org/release/Blender4.0/blender-4.0.0-linux-x64.tar.xz
    !mkdir -p "{DRIVE_ROOT}/EXODUS_AI_MODELS"
    !tar -xf blender-4.0.0-linux-x64.tar.xz -C "{DRIVE_ROOT}/EXODUS_AI_MODELS/"
    !rm blender-4.0.0-linux-x64.tar.xz
    print("✓ Blender 4.0 installé")
else:
    print(f"✓ Blender 4.0 déjà présent: {BLENDER_PATH}")

## 🔍 Cellule 2: Configuration Blender Portable

In [ ]:
# Vérification Blender
import subprocess

result = subprocess.run([BLENDER_PATH, "--version"], capture_output=True, text=True)
print("=== Blender Version ===")
print(result.stdout)

# Test Blender Python
test_script = '''
import bpy
print(f"Blender: {bpy.app.version_string}")
print(f"Python: {bpy.app.version[0]}.{bpy.app.version[1]}")
'''

result = subprocess.run(
    [BLENDER_PATH, "--background", "--python-expr", test_script],
    capture_output=True, text=True
)
print("\n=== Blender Python ===")
print(result.stdout)

## 📂 Cellule 3: Chargement et Validation des Inputs

In [ ]:
from pathlib import Path
import json

# Chemins à vérifier
paths_to_check = [
    ("Unit Root", UNIT_ROOT),
    ("Codebase", CODEBASE),
    ("IN_LOGISTICS", f"{UNIT_ROOT}/IN_LOGISTICS"),
    ("OUT_EQUIPPED", f"{UNIT_ROOT}/OUT_EQUIPPED"),
    ("Props Library", f"{UNIT_ROOT}/IN_LOGISTICS/props_library"),
]

print("=== Vérification Chemins ===")
for name, path in paths_to_check:
    exists = Path(path).exists()
    status = "✓" if exists else "✗"
    print(f"{status} {name}: {path}")

In [ ]:
# Lister les fichiers d'input
import os

in_logistics = f"{UNIT_ROOT}/IN_LOGISTICS"

print("\n=== Contenu IN_LOGISTICS ===")
if Path(in_logistics).exists():
    for item in os.listdir(in_logistics):
        item_path = Path(in_logistics) / item
        if item_path.is_dir():
            print(f"📁 {item}/")
            for sub in os.listdir(item_path):
                print(f"   └── {sub}")
        else:
            print(f"📄 {item}")
else:
    print("(dossier vide ou non créé)")

In [ ]:
# Valider le PRODUCTION_PLAN.JSON si présent
plan_path = f"{UNIT_ROOT}/IN_LOGISTICS/PRODUCTION_PLAN.JSON"

if Path(plan_path).exists():
    with open(plan_path, 'r') as f:
        plan = json.load(f)
    
    print("=== PRODUCTION_PLAN.JSON ===")
    print(f"Scenes: {len(plan.get('scenes', []))}")
    
    total_actions = 0
    props_needed = set()
    
    for scene in plan.get('scenes', []):
        actions = scene.get('props_actions', [])
        total_actions += len(actions)
        for action in actions:
            if 'prop_id' in action:
                props_needed.add(action['prop_id'])
    
    print(f"Total props_actions: {total_actions}")
    print(f"Props requis: {props_needed}")
else:
    print("⚠ PRODUCTION_PLAN.JSON non trouvé")
    print("Placez votre plan dans IN_LOGISTICS/PRODUCTION_PLAN.JSON")

## 🔩 Cellule 4: Test Unitaire Socketing sur UN Prop

In [ ]:
# Test du socketing sur un fichier blend de test
# Ce test nécessite un actor_animated.blend dans IN_LOGISTICS

ACTOR_BLEND = f"{UNIT_ROOT}/IN_LOGISTICS/actor_animated.blend"

if not Path(ACTOR_BLEND).exists():
    print("⚠ actor_animated.blend non trouvé")
    print("Placez votre fichier .blend animé dans IN_LOGISTICS/")
else:
    # Script Blender de test
    test_socketing_script = f'''
import bpy
import sys
sys.path.insert(0, "{CODEBASE}")

from socketing_engine import SocketingEngine

# Initialiser le moteur
engine = SocketingEngine(verbose=True)

# Trouver l'armature
armature = engine.find_armature()

if armature:
    print("\\n=== Bones disponibles ===")
    for bone in armature.data.bones[:20]:  # Limiter à 20
        print(f"  - {{bone.name}}")
    if len(armature.data.bones) > 20:
        print(f"  ... et {{len(armature.data.bones) - 20}} autres")
    
    print("\\n=== Sockets résolus ===")
    sockets = engine.list_available_sockets()
    for socket, bone in sockets.items():
        print(f"  {{socket}} -> {{bone}}")
else:
    print("Aucune armature trouvée!")
'''
    
    result = subprocess.run(
        [BLENDER_PATH, "--background", ACTOR_BLEND, "--python-expr", test_socketing_script],
        capture_output=True, text=True
    )
    
    print(result.stdout)
    if result.stderr:
        print("\nWarnings/Errors:")
        # Filtrer les warnings Blender courants
        for line in result.stderr.split('\n'):
            if 'WARN' in line or 'ERROR' in line or 'Error' in line:
                print(line)

## 🖼️ Cellule 5: Visualisation du Résultat

In [ ]:
# Générer une preview image si un blend existe
import subprocess
from IPython.display import Image, display

OUTPUT_DIR = f"{UNIT_ROOT}/OUT_EQUIPPED"
PREVIEW_PATH = f"{OUTPUT_DIR}/preview_test.png"

if Path(ACTOR_BLEND).exists():
    preview_script = f'''
import bpy

# Configuration rendu
scene = bpy.context.scene
scene.render.resolution_x = 800
scene.render.resolution_y = 600
scene.render.filepath = "{PREVIEW_PATH}"
scene.render.image_settings.file_format = 'PNG'

# Frame du milieu
mid_frame = (scene.frame_start + scene.frame_end) // 2
scene.frame_set(mid_frame)

# Rendu OpenGL (viewport)
bpy.ops.render.opengl(write_still=True)
print(f"Preview saved: {PREVIEW_PATH}")
print(f"Frame: {{mid_frame}}")
'''
    
    result = subprocess.run(
        [BLENDER_PATH, "--background", ACTOR_BLEND, "--python-expr", preview_script],
        capture_output=True, text=True
    )
    
    print(result.stdout)
    
    if Path(PREVIEW_PATH).exists():
        print("\n=== Preview ===")
        display(Image(filename=PREVIEW_PATH))
    else:
        print("Preview non générée")
else:
    print("Pas de fichier blend pour générer la preview")

## 📤 Cellule 6: Export Test

In [ ]:
# Test d'export Alembic
if Path(ACTOR_BLEND).exists():
    export_script = f'''
import bpy
import sys
sys.path.insert(0, "{CODEBASE}")

from final_baker import bake_and_export, get_export_stats, validate_export_ready

# Stats
stats = get_export_stats()
print("=== Export Stats ===")
for key, value in stats.items():
    print(f"  {{key}}: {{value}}")

# Validation
is_valid, issues = validate_export_ready()
print(f"\\nExport ready: {{is_valid}}")
if issues:
    print("Issues:")
    for issue in issues:
        print(f"  - {{issue}}")

# Test export (10 frames seulement)
if is_valid:
    test_abc = "{OUTPUT_DIR}/test_export.abc"
    frame_start = bpy.context.scene.frame_start
    bake_and_export(test_abc, frame_start, frame_start + 10)
'''
    
    result = subprocess.run(
        [BLENDER_PATH, "--background", ACTOR_BLEND, "--python-expr", export_script],
        capture_output=True, text=True
    )
    
    print(result.stdout)
    
    test_abc_path = f"{OUTPUT_DIR}/test_export.abc"
    if Path(test_abc_path).exists():
        size = Path(test_abc_path).stat().st_size
        print(f"\n✓ Test export réussi: {test_abc_path}")
        print(f"  Taille: {size / 1024:.1f} KB")
else:
    print("Pas de fichier blend pour tester l'export")

## 🚀 Cellule 7: Dry Run Pipeline Complet

In [ ]:
# Test dry-run du script principal
!python {CODEBASE}/EXO_02_LOGISTICS.py --help

In [ ]:
# Exemple de dry-run (validation chemins sans exécution)
# Décommenter et adapter les noms de fichiers

# !python {CODEBASE}/EXO_02_LOGISTICS.py \
#     --drive-root {DRIVE_ROOT} \
#     --actor-blend actor_animated.blend \
#     --production-plan PRODUCTION_PLAN.JSON \
#     --dry-run -v

---
## ✅ Checklist Debug

- [ ] Blender 4.0 installé et fonctionnel
- [ ] Chemins vérifiés
- [ ] actor_animated.blend présent
- [ ] PRODUCTION_PLAN.JSON valide
- [ ] Socketing résout les bones
- [ ] Export Alembic fonctionne
- [ ] Dry-run passe